# Задание 5

## 1. ContextTimer(0.3 балла)
Напишите менеджер контекста, который позволит засекать время выполнения блока кода с помощью конструкции with и выводить это время на экран по выходу из блока. Пример использования:

<code>
with Timer ():
    do_some_long_stuff()
</code>

Рекомендации: используйте стандартную библиотеку для работы со временем.

Протестируйте себя.

In [1]:
from time import time

class Timer:
    def __init__(self):
        self.start = None
        
    def __enter__(self):
        self.start = time()
        
    def __exit__(self, type, value, traceback):
        end = time()
        print('Time elapsed {}s'.format(end-self.start))

In [2]:
import wget

with Timer():
    URL = 'https://ia800303.us.archive.org/10/items/007474/007474_512kb.mp4'
    wget.download(URL, out='The-Fifth-Element-(1997).mpeg')

Time elapsed 5.560242176055908s


In [3]:
with Timer():
    print('Hello')

Hello
Time elapsed 6.175041198730469e-05s


## 2.Transaction (0.4 балла)
Вам необходимо написать менеджер контекстов, который позволит безопасно работать с транзакциями. Напишите класс Storage, в котором будут храниться какие-то данные в виде словаря. Эти данные должны быть закрытыми и их можно только читать через операцию []. У этого класса должен быть метод edit, который возвращает менеджер контекста, позволяющий редактировать исходный объект (опять же через []). При этом результаты редактирования записываются в исходный объект только если весь блок выполнился успешно. Пример использования:

<code>
s = Storage()
with s.edit() as se :
    se["a"] = 1
    may_be_an_exception_here()
</code>

Протестируйте себя.

In [4]:
class Storage:
    class ContextManager:
        def __init__(self, storage):
            self.storage = storage
        
        def __setitem__(self, key, value):
            self.storage._data[key] = value
            
        def __enter__(self):
            self.data_dict_copy = dict(self.storage._data)
            return self.storage._data
            
        def __exit__(self, type_, value_, traceback_):
            if (type_ is not None or value_ is not None or traceback_ is not None):
                self.storage._data = self.data_dict_copy               
            
    def __init__(self):
        self._data = {'key1': 'data1', 'key2': 'data2', 'key3': 'data3'}
        self._context_manager = self.ContextManager(self)
    
    def __getitem__(self, key):
        return self._data[key]
    
    def edit(self):
        return self._context_manager

In [5]:
s = Storage()
with s.edit() as se:
    se['a'] = 'b'
print(s._data)

{'key1': 'data1', 'key2': 'data2', 'key3': 'data3', 'a': 'b'}


In [6]:
s = Storage()
try:
    with s.edit() as se:
        se['a'] = 'c'
        raise NotImplementedError()
except:
    print(s._data)

{'key1': 'data1', 'key2': 'data2', 'key3': 'data3'}


## Phone numbers (0.3)
Напишите регулярное выражение для распознавания телефонных номеров и протестируйте себя. Номера, которые должны распознаваться:
* 3-22-46
* +71239513749
* 71239513749
* 1239513749
* +7(123)-951-37-49
* +7(123)9513749
* +7-123-9513749
* +7-123-951-37-49

* 7(123)-951-37-49
* 7(123)9513749
* 7-123-9513749
* 7-123-951-37-49

* (123)-951-37-49
* (123)9513749
* 123-9513749
* 123-951-37-49

In [7]:
first = r'\d{1,2}-\d{1,2}-\d{1,2}'
second = r'(\+7|7){0,1}-?((?=\()\(\d{3}\)|\d{3})-?\d{3}-?\d{2}-?\d{2}'
pattern = r'^' + first + r'|' + second + r'$'

In [8]:
pattern

'^\\d{1,2}-\\d{1,2}-\\d{1,2}|(\\+7|7){0,1}-?((?=\\()\\(\\d{3}\\)|\\d{3})-?\\d{3}-?\\d{2}-?\\d{2}$'

In [9]:
numbers = """3-22-46
+71239513749
71239513749
1239513749
+7(123)-951-37-49
+7(123)9513749
+7-123-9513749
+7-123-951-37-49
7(123)-951-37-49
7(123)9513749
7-123-9513749
7-123-951-37-49
(123)-951-37-49
(123)9513749
123-9513749
123-951-37-49"""

numbers = numbers.split('\n')

In [10]:
numbers

['3-22-46',
 '+71239513749',
 '71239513749',
 '1239513749',
 '+7(123)-951-37-49',
 '+7(123)9513749',
 '+7-123-9513749',
 '+7-123-951-37-49',
 '7(123)-951-37-49',
 '7(123)9513749',
 '7-123-9513749',
 '7-123-951-37-49',
 '(123)-951-37-49',
 '(123)9513749',
 '123-9513749',
 '123-951-37-49']

In [11]:
import re
regex = re.compile(pattern)

In [12]:
for phone in numbers:
    if regex.match(phone):
        print(phone, 'matches pattern')
    else:
        print(phone, "doesn't matches pattern")

3-22-46 matches pattern
+71239513749 matches pattern
71239513749 matches pattern
1239513749 matches pattern
+7(123)-951-37-49 matches pattern
+7(123)9513749 matches pattern
+7-123-9513749 matches pattern
+7-123-951-37-49 matches pattern
7(123)-951-37-49 matches pattern
7(123)9513749 matches pattern
7-123-9513749 matches pattern
7-123-951-37-49 matches pattern
(123)-951-37-49 matches pattern
(123)9513749 matches pattern
123-9513749 matches pattern
123-951-37-49 matches pattern
